Assume the data has been loaded from 
http://files.grouplens.org/datasets/movielens/ml-20m.zip and unpacked in data/ml-20m

In [1]:
import scipy.io as sio
import pandas as pd
from scipy.sparse import dok_matrix
from random import sample
import random
random.seed(123)

In [2]:
#read movies and tags
movies = pd.read_csv("data/ml-20m/movies.csv", quotechar='"')
tags = pd.read_csv("data/ml-20m/tags.csv", quotechar='"')
ratings = pd.read_csv("data/ml-20m/ratings.csv", quotechar='"')

In [72]:
### ratings.mtx
# movieId => movieRow
movie_dict = dict((m, i) for (i, m) in enumerate(movies['movieId']))
# userId => userRow
user_dict = dict((u, i) for (i, u) in enumerate(ratings['userId'].unique()))

In [4]:
nm, nr, nu = len(movies), len(ratings), len(user_dict)

In [76]:
nu, nm

(138493, 27278)

In [ ]:
# def get_rating_matrix(ratings, samples, save_to=None):
#     """
#         get (and write) rating matrix from long format 
#     """
#     mat = dok_matrix((nu, nm))
#     print save_to, 'samples length =', len(samples)

#     for t, i in enumerate(samples):
#         if t % 500000 == 0: print t
#         uid = user_dict[ratings['userId'][i]]
#         mid = movie_dict[ratings['movieId'][i]]
#         rating = ratings['rating'][i]
#         mat[uid,mid] = rating
#     if save_to is not None:
#         sio.mmwrite(save_to, mat)
#         print "done writing", save_to
#     return mat

In [87]:
def get_rating_matrix_from_sf(sf, save_to=None):
    """
        get (and write) rating matrix from long format 
    """
    mat = dok_matrix((nu, nm))
    n = len(sf)
    print 'save %s  samples length = %s' %(save_to, n)

    for i in xrange(n):
        if i % 10000 == 0: print i
        u, m = sf[i]['userId'], sf[i]['movieId']
        mat[u,m] = sf[i]['rating']
    if save_to is not None:
        sio.mmwrite(save_to, mat)
        print "done writing", save_to
    return mat

In [ ]:
def get_rating_sf(ratings, samples, save_to=None):
    sf = SFrame(ratings.ix[samples])
    sf['userId'] = sf['userId'].apply(lambda uid: user_dict[uid])
    sf['movieId'] = sf['movieId'].apply(lambda mid: movie_dict[mid])
    if save_to is not None:
        print "saving sframe to", save_to
        sf.save(save_to)
    return sf

In [79]:
def save_ratings_splits(ratings, train_ids, valid_ids, test_ids, name):
    valid_sf = get_rating_sf(ratings, valid_ids, save_to="data/%s_validate.sframe"%name)
    test_sf = get_rating_sf(ratings, test_ids, save_to="data/%s_test.sframe"%name)
    train_sf = get_rating_sf(ratings, train_ids, save_to="data/%s_train.sframe"%name)
    
    valid_mtx = get_rating_matrix_from_sf(valid_sf, "data/%s_valid.mtx"%name)
    test_mtx = get_rating_matrix_from_sf(test_sf, "data/%s_test.mtx"%name)
    train_mtx = get_rating_matrix_from_sf(train_sf, "data/%s_train.mtx"%name)

In [53]:
# def save_ratings_splits_mtx(ratings, train_ids, valid_ids, test_ids, name):
#     get_rating_matrix(ratings, valid_ids, save_to="data/%s_validate.mtx"%name)
#     get_rating_matrix(ratings, test_ids, save_to="data/%s_test.mtx"%name)
#     get_rating_matrix(ratings, train_ids, save_to="data/%s_train.mtx"%name)

In [80]:
# given list X, randomly split data into training, validation and test set
def sample_split(n, n_test=None, n_validate=None):
    n_test = n/5 if n_test is None else n_test
    n_validate = n/5 if n_validate is None else n_validate

    samples = sample(range(n), n_test + n_validate)
    test_ids = samples[0:n_test]
    validate_ids = samples[n_test:]
    train_ids = list(set(range(n)) - set(test_ids) - set(validate_ids))

    return train_ids, validate_ids, test_ids

In [ ]:
def debug_split():
    # smaller split for debugging 
    debug_size = nr/20
    debug_sample = sample(range(nr), debug_size)
    train_ids, valid_ids, test_ids = sample_split(debug_size)
    
    get_debug = lambda ids: map(lambda i: debug_sample[i], ids)
    
    train_ids = get_debug(train_ids)
    valid_ids = get_debug(valid_ids)
    test_ids = get_debug(test_ids)

    save_ratings_splits(ratings,  train_ids, valid_ids, test_ids, 'ratings_debug')
#     save_ratings_splits_mtx(ratings, train_ids, valid_ids, test_ids, 'ratings_debug')
    
    return train_ids, valid_ids, test_ids

In [ ]:
debug_train_ids, debug_valid_ids, debug_test_ids = debug_split()

In [89]:
def normal_split():
    train_ids, valid_ids, test_ids = sample_split(nr)
    save_ratings_splits(ratings, train_ids, valid_ids, test_ids, 'ratings')
#     save_ratings_splits_mtx(ratings, train_ids, valid_ids, test_ids, 'ratings')
    return train_ids, valid_ids, test_ids

In [90]:
train_ids, valid_ids, test_ids = normal_split()

saving sframe to data/ratings_validate.sframe
saving sframe to data/ratings_test.sframe
saving sframe to data/ratings_train.sframe
save data/ratings_valid.mtx  samples length = 4000052

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored
ERROR: 


0
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 267, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/local/Cellar/python/2.7.3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1043, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/local/Cellar/python/2.7.3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1003, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/local/Cellar/python/2.7.3/Frameworks/Python.framework/Versions/2.7/lib/py

Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
rating_matrix = get_rating_matrix(ratings, range(len(ratings)))

In [50]:
genres = movies['genres'].map(lambda x: set(x.split('|')))
unique_genres = reduce(lambda a,b: a|b, genres, set())
unique_genres = filter(lambda x: x!='(no genres listed)', unique_genres)
ng = len(unique_genres)

# genreId => genreRow
genres_dict = dict((t, i) for (i, t) in enumerate(unique_genres))
# tagId => tagRow
tags_id_dict = dict((t, i) for (i, t) in enumerate(tags['tag'].unique()))
nt = len(tags_id_dict)

In [51]:
#FIXME refactor this 
def get_movie_matrix():
    mat = dok_matrix((nm, ng + nt))

    for i, gs in enumerate(genres):
        for g in gs:
            if g in genres_dict:
                mat[i,genres_dict[g]] = 1

    for i in xrange(len(tags)):
        mid = movie_dict[tags['movieId'][i]]
        t = tags['tag'][i]
        ti = ng + tags_id_dict[t]
        mat[mid, ti] += 1    
    sio.mmwrite('data/movies.mtx', mat)
    return mat

In [52]:
get_movie_matrix()

<27278x38663 sparse matrix of type '<type 'numpy.float64'>'
	with 254540 stored elements in Dictionary Of Keys format>

In [32]:
import graphlab as gl
from graphlab import SFrame
from graphlab import SGraph

In [33]:
def prefix(p):
    return lambda x: "%s%s"%(p,x)

def remove(c):
    def r(x):
        del x[c]
        return x
    return r

In [37]:
# user_ids = range(len(user_dict))
# movies_ids = range(len(movies_dict))
# vertices = SFrame({
#         'id': map(prefix('m'), movies_ids),
#         'factors': map(lambda _: np.ones(5), movies_ids), #FIXME
#         'features': map(lambda _:{}, movies_ids),  #FIXME
#         'user':  map(lambda _: 0, movies_ids)
#     }).append(SFrame({
#         'id': map(prefix('u'), user_ids), 
#         'factors': map(lambda _: np.ones(5), user_ids), 
#         'features': map(lambda _:{}, user_ids),
#         'user': map(lambda _: 1, user_ids)
#     }))

In [44]:
# def create_graph(vertices, ratings, suffix):
#     ratings_sf = SFrame(ratings)
#     ratings_sf['userId'] = ratings_sf['userId'].apply(prefix('u'))
#     ratings_sf['movieId'] = ratings_sf['movieId'].apply(prefix('m'))
#     s = SGraph().add_vertices(vertices, vid_field='id')\
#         .add_edges(ratings_sf, src_field='userId', dst_field='movieId')
#     s.save('data/ratings%s_train.sgraph' % suffix)

In [46]:
create_graph(vertices, ratings.ix[debug_train_ids], '_debug')

In [47]:
create_graph(vertices, ratings.ix[train_ids], '')

In [ ]:
# FIXME finish this
# split for cold start
create_graph(vertices, ratings_train_cs, 'cs')
create_graph(vertices, ratings_train_cs_debug, 'cs_debug')